In [1]:
! pip install --upgrade --quiet  weaviate-client

In [3]:
import weaviate

#seems like weaviate_client invokes access to collections, while client is truly just the client connection
#docker run -p 8081:8080 -p 50051:50051 -e ENABLE_MODULES=text2vec-ollama cr.weaviate.io/semitechnologies/weaviate:1.25.1
#make sure the above is running in docker

weaviate_client = weaviate.Client("http://localhost:8081")
client = weaviate.connect_to_local("localhost","8081")#v4


/home/jonot480/miniconda3/envs/myenv/lib/python3.12/site-packages/weaviate/warnings.py:162: DeprecationWarning: Dep016: Python client v3 `weaviate.Client(...)` connections and methods are deprecated. Update
            your code to use Python client v4 `weaviate.WeaviateClient` connections and methods.

            For Python Client v4 usage, see: https://weaviate.io/developers/weaviate/client-libraries/python
            For code migration, see: https://weaviate.io/developers/weaviate/client-libraries/python/v3_v4_migration
            
  warnings.warn(


In [53]:
client.collections.delete_all()

In [4]:
from langchain_community.retrievers import (
    WeaviateHybridSearchRetriever,
)

from weaviate.classes.config import Configure

index_name="JonsTest"
#index_name="LangChain_9ffcc707dbbf428d8c7e147134476d07"


In [5]:
client.collections.create(
    index_name,
    vectorizer_config=Configure.Vectorizer.text2vec_ollama(
        model="nomic-embed-text",    
        api_endpoint="http://host.docker.internal:11434",
    ),

)

UnexpectedStatusCodeError: Collection may not have been created properly.! Unexpected status code: 422, with response body: {'error': [{'message': 'class name JonsTest already exists'}]}.

In [7]:
collection = client.collections.get(index_name)

response = collection.query.fetch_objects()
docs = []

for o in response.objects:
     print(o.properties["text"])
     #docs.append(o.properties["text"])
     #docs.append(o)
     #print(o)


Most people think of nerds as quiet, diffident people. In ordinary social situations they are — as quiet and diffident as the star quarterback would be if he found himself in the middle of a physics symposium. And for the same reason: they are fish out of water. But the apparent diffidence of nerds is an illusion due to the fact that when non-nerds observe them, it's usually in ordinary social situations. In fact some nerds are quite fierce. Effective dated 4/2/2024. 

The fierce nerds are a small but interesting group. They are as a rule extremely competitive — more competitive, I'd say, than highly competitive non-nerds. Competition is more personal for them. Partly perhaps because they're not emotionally mature enough to distance themselves from it, but also because there's less randomness in the kinds of competition they engage in, and they are thus more justified in taking the results personally.

Fierce nerds also tend to be somewhat overconfident, especially when young. It might

In [1]:
from langchain_core.documents import Document

docs = []

for item in collection.iterator():
    #print(item.uuid, item.properties.get("text"))
    #docs.append([,item.properties.get("text"),item.properties])
    docs.append(Document(uuid=item.uuid,page_content=item.properties.get("text"),metadata=item.properties))

print(docs)


NameError: name 'collection' is not defined

In [98]:

retriever = WeaviateHybridSearchRetriever(
    client=weaviate_client,
    index_name=index_name,
    text_key="text",
    attributes=[],
    create_schema_if_missing=True
)


In [7]:

from langchain_community.document_loaders import TextLoader

from langchain.text_splitter import CharacterTextSplitter

#loader = TextLoader("/home/jonot480/Documents/jon craig.txt")
#doc_text = loader.load()

loader = TextLoader("/home/jonot480/Documents/paulgraham.txt")
documents = loader.load()
text_splitter = CharacterTextSplitter(chunk_size=100,chunk_overlap=0)
docs = text_splitter.split_documents(documents)


Created a chunk of size 472, which is longer than the specified 100
Created a chunk of size 440, which is longer than the specified 100
Created a chunk of size 243, which is longer than the specified 100
Created a chunk of size 235, which is longer than the specified 100
Created a chunk of size 518, which is longer than the specified 100
Created a chunk of size 408, which is longer than the specified 100
Created a chunk of size 1198, which is longer than the specified 100
Created a chunk of size 488, which is longer than the specified 100
Created a chunk of size 461, which is longer than the specified 100
Created a chunk of size 416, which is longer than the specified 100
Created a chunk of size 218, which is longer than the specified 100
Created a chunk of size 328, which is longer than the specified 100
Created a chunk of size 395, which is longer than the specified 100


In [99]:
#retriever.add_documents(doc_text)
retriever.add_documents(docs)

['7deba3de-84df-4e7a-97dc-6bfbd404b2cf',
 '223728c0-d9c1-486d-96a6-cecbde138436',
 '246bd63f-8458-41ca-8f76-4940623a2080',
 '91f44a84-3159-4206-b00e-2c8770fadc70',
 '0e6b74e3-f574-4184-b7c2-fa7c7c5c6d8b',
 '5c31e2c4-b483-47f9-a863-a71da7a77f29',
 '5dd7a2f8-35e0-44b9-b651-5ed36ce2c446',
 'f34c809b-ddfb-4ba0-949f-aed978a9c857',
 '2e8868e7-b002-4321-b1f9-5238724a3d5e',
 'f8fe3d4b-fc52-425d-97ab-d213b446198c',
 '9de67480-7409-4978-b1f7-ad1f15e5fc96',
 'fa06f2ba-457c-49fb-be75-c906be01ad1f',
 '73451753-3202-4ce4-8435-a857f4558b7b',
 'ba9a563c-9fd6-4e93-8fb5-30db06df51f5',
 '0b163026-2d5b-436d-9b39-90b08c3ec120',
 '8b747d79-8b58-4aeb-9b7d-fd442c31e8a7',
 'e15e5222-29bb-43ed-91d7-4441165c97f8',
 '6d264124-c402-4683-a8bd-84abfbf2f785',
 'ae7d2f02-f754-42a9-9b0d-6ecb7626b9a7',
 'af9aa4a4-fb5b-4731-85a1-71c60cf48404',
 '183f2f2f-d793-4ca4-b217-630788b24d18',
 '36fe218c-dce3-4afa-8da8-d3f0e493fcf1',
 'a95afff8-e8cd-4bee-adb4-e0ef9d85d36c',
 '89fcbee5-6098-4d1c-9bec-fe0dc3430eaa',
 '047a5236-9496-

In [105]:
result = retriever.invoke(
    "effective date",
    score=True    
)

result = {
            "distances": [d.metadata.get("_additional").get('score') for d in result],
            "documents": [d.page_content for d in result],
            "metadatas": [d.metadata for d in result]
        }
print(result)

{'distances': ['0.91464955', '0.91464955', '0.91464955', '0.91464955'], 'documents': ['The idea that there could be fierce nerds is an unfamiliar one not just to many normal people but even to some young nerds. Charlie is good at math.  Especially early on, nerds spend so much of their time in ordinary social situations and so little doing real work that they get a lot more evidence of their awkwardness than their power. So there will be some who read this description of the fierce nerd and realize "Hmm, that\'s me." And it is to you, young fierce nerd, that I now turn.\n\nI have some good news, and some bad news. The good news is that your fierceness will be a great help in solving difficult problems like finding the effective date. And not just the kind of scientific and technical problems that nerds have traditionally solved. As the world progresses, the number of things you can win at by getting the right answer increases. Recently getting rich became one of them: 7 of the 8 riches

In [56]:
from pprint import pprint
#pprint(result)
#print(result)
#print([d.metadata.get("_additional") for d in result])
#print([d.metadata.get("_additional") for d in result])
#print([[d.metadata.get("_additional").get('score') for d in result]])
#print([d.page_content for d in result])
print([d.metadata for d in result])

print(client.collections.list_all())


[{'_additional': {'explainScore': '\nHybrid (Result Set keyword,bm25) Document 04e67841-d6fa-4139-970d-49f375bf7e39: original score 0.98960793, normalized score: 0.5 - \nHybrid (Result Set vector,hybridVector) Document 04e67841-d6fa-4139-970d-49f375bf7e39: original score 0.34004956, normalized score: 0.5', 'score': '1'}}, {'_additional': {'explainScore': '\nHybrid (Result Set keyword,bm25) Document 4e546770-6e26-4c96-b0c2-d848afcf6ec3: original score 0.95391834, normalized score: 0.46945986 - \nHybrid (Result Set vector,hybridVector) Document 4e546770-6e26-4c96-b0c2-d848afcf6ec3: original score 0.3292644, normalized score: 0.46984702', 'score': '0.93930686'}}, {'_additional': {'explainScore': '\nHybrid (Result Set keyword,bm25) Document 58a0dd76-1140-4eb2-8c4b-a68cb8d1569a: original score 0.4053014, normalized score: 0 - \nHybrid (Result Set vector,hybridVector) Document 58a0dd76-1140-4eb2-8c4b-a68cb8d1569a: original score 0.31884634, normalized score: 0.44072038', 'score': '0.44072038

In [12]:
result = {
    "distances": [[d.metadata.get("score") for d in result]],
    "documents": [[d.page_content for d in result]],
    "metadatas": [[d.metadata for d in result]],
}

In [13]:
print(result)

{'distances': [[None, None, None, None]], 'documents': [['I have some good news, and some bad news. The good news is that your fierceness will be a great help in solving difficult problems like finding the effective date. And not just the kind of scientific and technical problems that nerds have traditionally solved. As the world progresses, the number of things you can win at by getting the right answer increases. Recently getting rich became one of them: 7 of the 8 richest people in America are now fierce nerds.', "If you do choose the ambitious route, you'll have a tailwind behind you with an effective date of 4/6/2023. There has never been a better time to be a nerd. In the past century we've seen a continuous transfer of power from dealmakers to technicians — from the charismatic to the competent — and I don't see anything on the horizon that will end it. At least not till the nerds end it themselves by bringing about the singularity.", "Most people think of nerds as quiet, diffid